In [89]:
import os
import gc
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
import tensorflow as tf
# 001
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121, InceptionV3, EfficientNetB0, Xception, MobileNetV2
from tensorflow.keras import models, layers, optimizers, callbacks
from tensorflow.keras import backend as K

# 002
# from keras._tf_keras.keras.preprocessing.image import  ImageDataGenerator
# from keras._tf_keras.keras.applications import VGG16, ResNet50, DenseNet121, InceptionV3, EfficientNetB0, Xception, MobileNetV2
# from keras._tf_keras.keras import models, layers, optimizers, callbacks
# from keras._tf_keras.keras import backend as K


print("Todas as bibliotecas foram importadas com sucesso!")

Todas as bibliotecas foram importadas com sucesso!


In [90]:
# Criando função para calcular as métricas

def get_metrics(y_true, y_pred):
    vn, fp, fn, vp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = (vp + vn) / (vp + fp + fn + vn)  
    recall = vp / (vp + fn)
    specificity = vn / (vn + fp)
    precision = vp / (vp + fp)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred)

    return {
        'accuracy': accuracy,
        'specificity': specificity,
        'recall': recall,
        'precision': precision,
        'f1-score': f1,
        'kappa': kappa,
        'auc-roc': auc_roc
    }


In [91]:
# Criando função para seleção de esquema de cor 
def convert_color_scale(image, scale):
    if scale == 'hsv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    elif scale == 'rgb':
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    elif scale == 'grayscale':
        # Converter para escala de cinza e replicar para 3 canais
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return cv2.merge([gray, gray, gray])
    elif scale == 'lab':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    elif scale == 'luv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
    elif scale == 'xyz':
        return cv2.cvtColor(image, cv2.COLOR_BGR2XYZ)
    else:
        raise ValueError("Escala de cor não suportada.")

In [92]:
# Carregamento e pré-processamento de imagens com escolha de escala de cor

def load_images(folder, color_scale, img_extensions):
    images = []
    for filename in os.listdir(folder):
        if any(filename.lower().endswith(ext) for ext in img_extensions):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (224, 224))  # Ajuste o tamanho conforme necessário
                img = convert_color_scale(img, color_scale)  # Converta para a escala de cor desejada
                # Se a imagem estiver em escala de cinza, expanda as dimensões
                if color_scale == 'grayscale':
                    img = np.expand_dims(img, axis=-1)  # Adiciona uma dimensão de canal
                images.append(img)

    return np.array(images)


In [93]:
# Defina as pastas de dados
data_dir = r"C:\Users\andre\Pictures\OvarianCancer"

normal_dir = os.path.join(data_dir, 'Non_Cancerous')
endometri = os.path.join(data_dir, 'Endometri')
Clear_Cell = os.path.join(data_dir, 'Clear_Cell')
Mucinous = os.path.join(data_dir, 'Mucinous')
Serous = os.path.join(data_dir, 'Serous')

img_extensions = ['.jpg', '.jpeg', '.png']

if os.listdir(normal_dir):print("pasta acesada, na pasta normal ha itens")
if os.listdir(endometri):print("pasta acessada, na pasta endometri ha itens")
if os.listdir(Clear_Cell):print("pasta acessada, na pasta Clear_Cell ha itens")
if os.listdir(Mucinous):print("pasta acessada, na pasta Mucinous ha itens")
if os.listdir(Serous):print("pasta acessada, na pasta Serous ha itens")



pasta acesada, na pasta normal ha itens
pasta acessada, na pasta endometri ha itens
pasta acessada, na pasta Clear_Cell ha itens
pasta acessada, na pasta Mucinous ha itens
pasta acessada, na pasta Serous ha itens


In [94]:
# Criar um DataFrame para armazenar os resultados
columns = ['Modelo', 'Acuracia', 'Sensibilidade', 'Especificidade', 'F-Score', 'AUC-ROC']
df_metrics = pd.DataFrame(columns=columns)

# Definir o caminho para salvar o melhor modelo
model_checkpoint_path = r"C:\Users\andre\Documents\atividades\estudos_opencv\processamento de imagens\\best_model.h5"

In [ ]:
# Iterar sobre os modelos
def model_execution(model_name,pasta_com_nonCancer):

    # Carregamento de imagens e conversão para XYZ
    normal_images = load_images(normal_dir, 'rgb',img_extensions)
    cancer_images = load_images(pasta_com_nonCancer, 'rgb',img_extensions)

    # Rótulos para imagens (0 para normal, 1 para câncer)
    normal_labels = np.zeros(normal_images.shape[0])
    cancer_labels = np.ones(cancer_images.shape[0])

    # Concatenar imagens e rótulos
    all_images = np.concatenate([normal_images, cancer_images], axis=0)
    all_labels = np.concatenate([normal_labels, cancer_labels], axis=0)

    # Dividir o conjunto de dados em treino e teste (80/20)
    X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

    # Escolha entre 5 tipos de conversão de escala de cor aqui (por exemplo, normalização)
    X_train = X_train / 255.0
    X_test = X_test / 255.0

    # Escolher o modelo de CNN
    if model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'MobileNetV2':
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'InceptionV3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'EfficientNetB0':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'Xception':
        base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'VisionTransformer':
        base_model = VitB16(image_size=(224, 224), weights='imagenet', include_top=False)
    else:
        raise ValueError("Modelo de CNN não suportado.")

    # Construir o modelo
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compilar o modelo
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Configurar o retorno de chamada ModelCheckpoint
#     checkpoint = callbacks.ModelCheckpoint(model_checkpoint_path,
#                                            monitor='val_accuracy',  # Métrica a ser monitorad
#                                            mode='max',               # Salvar o modelo com a maior precisão
#                                            verbose=1)

    # Treinar o modelo
    model.fit(X_train, y_train, epochs=50, batch_size=20, validation_split=0.2)  #  16 <<====

    # Avaliar o modelo
    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Calcular métricas
    metrics = get_metrics(y_test, y_pred_binary)

    # Adicionar as métricas ao DataFrame
    metrics['Modelo'] = model_name
    
    # Excluir o modelo atual para liberar memória da GPU   
    del all_images, normal_images, cancer_images, X_train, X_test, y_train, y_test, y_pred, y_pred_binary
    del all_labels, normal_labels, cancer_labels
    tf.keras.backend.clear_session()
    del model, base_model
    gc.collect()
    
    return metrics


In [96]:
chosen_model = 'VGG16'
#       ENDOMETRI
metrics = model_execution(chosen_model,endometri)
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics1 = pd.concat([df_metrics, temp_df], ignore_index=True)
#       Clear_Cell  
metrics = model_execution(chosen_model,Clear_Cell) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics2 = pd.concat([df_metrics, temp_df], ignore_index=True) # <==
#       Mucinous
metrics = model_execution(chosen_model,Mucinous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics3 = pd.concat([df_metrics, temp_df], ignore_index=True) # <==
#       Serous
metrics = model_execution(chosen_model,Serous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics4 = pd.concat([df_metrics, temp_df], ignore_index=True) # <==



chosen_model = 'ResNet50'
#       ENDOMETRI
metrics = model_execution(chosen_model,endometri)
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics1 = pd.concat([df_metrics1, temp_df], ignore_index=True)
#       Clear_Cell  
metrics = model_execution(chosen_model,Clear_Cell) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics2 = pd.concat([df_metrics2, temp_df], ignore_index=True) # <==
#       Mucinous
metrics = model_execution(chosen_model,Mucinous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics3 = pd.concat([df_metrics3, temp_df], ignore_index=True) # <==
#       Serous
metrics = model_execution(chosen_model,Serous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics4 = pd.concat([df_metrics4, temp_df], ignore_index=True) # <==



chosen_model = 'InceptionV3'
#       ENDOMETRI
metrics = model_execution(chosen_model,endometri)
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics1 = pd.concat([df_metrics1, temp_df], ignore_index=True)
#       Clear_Cell  
metrics = model_execution(chosen_model,Clear_Cell) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics2 = pd.concat([df_metrics2, temp_df], ignore_index=True) # <==
#       Mucinous
metrics = model_execution(chosen_model,Mucinous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics3 = pd.concat([df_metrics3, temp_df], ignore_index=True) # <==
#       Serous
metrics = model_execution(chosen_model,Serous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics4 = pd.concat([df_metrics4, temp_df], ignore_index=True) # <==



chosen_model = 'MobileNetV2'
#       ENDOMETRI
metrics = model_execution(chosen_model,endometri)
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics1 = pd.concat([df_metrics1, temp_df], ignore_index=True)
#       Clear_Cell  
metrics = model_execution(chosen_model,Clear_Cell) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics2 = pd.concat([df_metrics2, temp_df], ignore_index=True) # <==
#       Mucinous
metrics = model_execution(chosen_model,Mucinous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics3 = pd.concat([df_metrics3, temp_df], ignore_index=True) # <==
#       Serous
metrics = model_execution(chosen_model,Serous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics4 = pd.concat([df_metrics4, temp_df], ignore_index=True) # <==



chosen_model = 'EfficientNetB0'
#       ENDOMETRI
metrics = model_execution(chosen_model,endometri)
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics1 = pd.concat([df_metrics1, temp_df], ignore_index=True)
#       Clear_Cell  
metrics = model_execution(chosen_model,Clear_Cell) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics2 = pd.concat([df_metrics2, temp_df], ignore_index=True) # <==
#       Mucinous
metrics = model_execution(chosen_model,Mucinous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics3 = pd.concat([df_metrics3, temp_df], ignore_index=True) # <==
#       Serous
metrics = model_execution(chosen_model,Serous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics4 = pd.concat([df_metrics4, temp_df], ignore_index=True) # <==



print("\033[32m-------    ENDOMETRI e NON_CONCEROUS   ------\033[m ")
display(df_metrics1)


print("\033[32m-------    Clear_Cell e NON_CONCEROUS   ------\033[m ")
display(df_metrics2)

print("\033[32m-------    Mucinous e NON_CONCEROUS   ------\033[m ")
display(df_metrics3)

print("\033[32m-------    Serous e NON_CONCEROUS   ------\033[m ")
display(df_metrics4)

Epoch 1/50
7/7 [==============================] - 9s 715ms/step - loss: 0.7302 - accuracy: 0.5680 - val_loss: 0.6293 - val_accuracy: 0.6562
Epoch 2/50
7/7 [==============================] - 1s 180ms/step - loss: 0.6340 - accuracy: 0.6560 - val_loss: 0.5347 - val_accuracy: 0.7500
Epoch 3/50
7/7 [==============================] - 1s 180ms/step - loss: 0.5421 - accuracy: 0.7360 - val_loss: 0.4698 - val_accuracy: 0.7188
Epoch 4/50
7/7 [==============================] - 1s 181ms/step - loss: 0.5505 - accuracy: 0.7040 - val_loss: 0.7338 - val_accuracy: 0.5312
Epoch 5/50
7/7 [==============================] - 1s 183ms/step - loss: 0.5099 - accuracy: 0.7440 - val_loss: 0.3981 - val_accuracy: 0.8438
Epoch 6/50
7/7 [==============================] - 1s 182ms/step - loss: 0.3109 - accuracy: 0.8800 - val_loss: 0.5339 - val_accuracy: 0.6875
Epoch 7/50
7/7 [==============================] - 1s 181ms/step - loss: 0.2463 - accuracy: 0.8960 - val_loss: 0.5373 - val_accuracy: 0.8438
Epoch 8/50
7/7 [====

c:\Users\andre\anaconda3\envs\TFenv\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  


Epoch 1/50
7/7 [==============================] - 5s 234ms/step - loss: 1.7494 - accuracy: 0.5276 - val_loss: 0.8211 - val_accuracy: 0.4375
Epoch 2/50
7/7 [==============================] - 1s 145ms/step - loss: 0.1043 - accuracy: 0.9764 - val_loss: 2.3416 - val_accuracy: 0.5625
Epoch 3/50
7/7 [==============================] - 1s 146ms/step - loss: 0.0439 - accuracy: 0.9843 - val_loss: 0.7292 - val_accuracy: 0.5625
Epoch 4/50
7/7 [==============================] - 1s 147ms/step - loss: 0.0469 - accuracy: 0.9843 - val_loss: 0.7092 - val_accuracy: 0.4688
Epoch 5/50
7/7 [==============================] - 1s 145ms/step - loss: 0.0663 - accuracy: 0.9764 - val_loss: 0.7531 - val_accuracy: 0.5625
Epoch 6/50
7/7 [==============================] - 1s 147ms/step - loss: 0.0086 - accuracy: 0.9921 - val_loss: 0.7436 - val_accuracy: 0.5625
Epoch 7/50
7/7 [==============================] - 1s 145ms/step - loss: 0.0121 - accuracy: 0.9921 - val_loss: 0.6974 - val_accuracy: 0.5000
Epoch 8/50
7/7 [====

,Modelo,Acuracia,Sensibilidade,Especificidade,F-Score,AUC-ROC
0,VGG16,0.925,1.000000,0.857143,0.926829,0.928571
1,ResNet50,0.475,1.000000,0.000000,0.644068,0.500000
2,InceptionV3,0.850,0.842105,0.857143,0.842105,0.849624
3,MobileNetV2,0.900,0.789474,1.000000,0.882353,0.894737
4,EfficientNetB0,0.525,0.052632,0.952381,0.095238,0.502506


-------    Clear_Cell e NON_CONCEROUS   ------ 


,Modelo,Acuracia,Sensibilidade,Especificidade,F-Score,AUC-ROC
0,VGG16,0.875,0.789474,0.952381,0.857143,0.870927
1,ResNet50,0.525,0.000000,1.000000,0.000000,0.500000
2,InceptionV3,0.875,0.842105,0.904762,0.864865,0.873434
3,MobileNetV2,0.750,0.631579,0.857143,0.705882,0.744361
4,EfficientNetB0,0.425,0.631579,0.238095,0.510638,0.434837


-------    Mucinous e NON_CONCEROUS   ------ 


,Modelo,Acuracia,Sensibilidade,Especificidade,F-Score,AUC-ROC
0,VGG16,0.775,0.842105,0.714286,0.780488,0.778195
1,ResNet50,0.550,0.052632,1.000000,0.100000,0.526316
2,InceptionV3,0.750,0.578947,0.904762,0.687500,0.741855
3,MobileNetV2,0.700,0.368421,1.000000,0.538462,0.684211
4,EfficientNetB0,0.525,0.315789,0.714286,0.387097,0.515038


-------    Serous e NON_CONCEROUS   ------ 


,Modelo,Acuracia,Sensibilidade,Especificidade,F-Score,AUC-ROC
0,VGG16,0.900,0.842105,0.952381,0.888889,0.897243
1,ResNet50,0.475,1.000000,0.000000,0.644068,0.500000
2,InceptionV3,0.875,0.842105,0.904762,0.864865,0.873434
3,MobileNetV2,0.900,0.789474,1.000000,0.882353,0.894737
4,EfficientNetB0,0.600,0.157895,1.000000,0.272727,0.578947
